In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime
import ast
import statistics
import sys
from timeit import default_timer as timer
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.interpolate import griddata
from scipy.stats import mannwhitneyu
from scipy.stats import skew
from scipy.stats import kruskal

## Merging all data

In [ ]:
# Reading all the files and concatenating

csv_file_directory = r"D:\Thesis Project\Thesis Project 2\Research Phase 4\Data\CAHR Data\CAHR Combined - Physiology Data"
files = os.listdir(csv_file_directory)

df_all = pd.DataFrame()

parameter_list = ['DateTime', 'RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                      'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']

for file in files:
    df_individual= pd.read_csv(os.path.join(csv_file_directory, file))
    df_individual_cleaned = df_individual.reindex(columns = parameter_list)
    
    filter_condition = (df_individual_cleaned['ICP'] > 100) | (df_individual_cleaned['ICP'] < 0) | \
                       (df_individual_cleaned['MAP'] > 200) | (df_individual_cleaned['MAP'] < 0) | \
                       (df_individual_cleaned['AMP'] > 40)
    df_individual_cleaned = df_individual_cleaned[~filter_condition] 
    
    df_all = pd.concat([df_all, df_individual_cleaned], axis=0, ignore_index=True)

## Converting to different resolution

In [ ]:
def excel_serial_to_datetime(serial):
    
    return datetime.datetime(1899, 12, 30) + datetime.timedelta(days=serial)

In [ ]:
def convert_parameter_resolution(df_main, resolution):
    
    df_all_process = df_main.copy()
    
    df_all_process['datetime'] = df_all_process['DateTime'].apply(excel_serial_to_datetime)
    data_resampled = df_all_process.set_index('datetime')
    data_resampled = data_resampled.resample(resolution).mean()
    
    return data_resampled

## Refining the data with respect to rSO2 artifacts

In [ ]:
def timestamps_with_rso2_artifacts(df_all):
    
    filter_condition = (df_all['RSO2_L'] < 20) | (df_all['RSO2_R'] < 20)
    filtered_timestamps = df_all[filter_condition]['DateTime']
    
    return filtered_timestamps

In [ ]:
filtered_timestamps = timestamps_with_rso2_artifacts(df_all)

df_pair = df_all[['DateTime', 'RAP', 'RSO2_R']]
df_pair = df_pair[~df_pair['DateTime'].isin(filtered_timestamps)]

In [ ]:
def cleaning_NIRS_measures_parameters(df_nirs_measures):
    
    filter_condition = (df_nirs_measures['RSO2_L'] < 20) | (df_nirs_measures['RSO2_R'] < 20)
    df_nirs_measures = df_nirs_measures[~filter_condition]
    
    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

    for param in NIRS_measures_list:
        df_nirs_measures = df_nirs_measures.loc[df_nirs_measures[param] != 0]

    return df_nirs_measures

In [ ]:
def cleaning_NIRS_measures_parameters_v2(df_nirs_measures, parameter):

    filter_condition = (df_nirs_measures['RSO2_L'] < 20) | (df_nirs_measures['RSO2_R'] < 20)
    df_nirs_measures = df_nirs_measures[~filter_condition]
    df_nirs_measures = df_nirs_measures.loc[df_nirs_measures[parameter] != 0]

    return df_nirs_measures

## Scatterplots (piece-wise linear regression)

In [ ]:
def scatter_plot_with_piece_wise_linear_regression(df_all, dependent_param, resolution, filtered_timestamps):
    
    warnings.filterwarnings('ignore')
     
    # Separating the columns that would be plotted
    df_pair = df_all[['DateTime', 'RAP', dependent_param]]
    
    # Declaring the NIRS measures
    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']
    # Checking if it's a NIRS measure and if yes, dropping the rows according to the threshold
    if dependent_param in NIRS_measures_list:
        df_pair = df_pair[~df_pair['DateTime'].isin(filtered_timestamps)]
        df_pair = df_pair.loc[df_pair[dependent_param] != 0]
        
    # Dropping the Nan values
    df_pair = df_pair.dropna()
    # Converting the resolution
    if resolution == 'M':
        df_pair_resolution = df_pair
    else:   
        df_pair_resolution = convert_parameter_resolution(df_pair, resolution)
    
    # Segmenting the data with respect to RAP thresholds
    breakpoints = [0, 0.4]

    segment1 = df_pair_resolution[df_pair_resolution['RAP'] < breakpoints[0]]
    segment2 = df_pair_resolution[(df_pair_resolution['RAP'] >= breakpoints[0]) & (df_pair_resolution['RAP'] <= breakpoints[1])]
    segment3 = df_pair_resolution[df_pair_resolution['RAP'] > breakpoints[1]]
    
    # Fitting linear regression models for each segment
    model1 = LinearRegression().fit(segment1[['RAP']], segment1[dependent_param])
    model2 = LinearRegression().fit(segment2[['RAP']], segment2[dependent_param])
    model3 = LinearRegression().fit(segment3[['RAP']], segment3[dependent_param])

    # Predict values for plotting
    x_range1 = np.linspace(segment1['RAP'].min(), segment1['RAP'].max(), 100).reshape(-1, 1)
    x_range2 = np.linspace(segment2['RAP'].min(), segment2['RAP'].max(), 100).reshape(-1, 1)
    x_range3 = np.linspace(segment3['RAP'].min(), segment3['RAP'].max(), 100).reshape(-1, 1)

    y_pred1 = model1.predict(x_range1)
    y_pred2 = model2.predict(x_range2)
    y_pred3 = model3.predict(x_range3)
    
    plt.figure(figsize=(10, 8))
    
    plt.scatter(df_pair_resolution['RAP'], df_pair_resolution[dependent_param], s=5, alpha=0.6)
    
    # Plot fitted lines
    plt.plot(x_range1, y_pred1, color='darkred', label='RAP < 0 Fit')
    plt.plot(x_range2, y_pred2, color='darkgreen', label='0 ≤ RAP ≤ 0.4 Fit')
    plt.plot(x_range3, y_pred3, color='#CCCC00', label='RAP > 0.4 Fit')

    # Add titles and labels
    title = 'Piece-wise Linear Regression with Two Thresholds Between RAP and ' + dependent_param
    plt.title(title)
    plt.xlabel('RAP')
    plt.ylabel(dependent_param)
    plt.legend(loc='upper left')
    
    save_image_directory = r"D:\Thesis Project\Thesis Project 2\Research Phase 4\Plots\Piece_wise_linear_regression\RAP_independent"
    save_image_name = resolution + ' RAP and ' + dependent_param
    plt.savefig(os.path.join(save_image_directory, save_image_name))
    
    plt.show()

    # Print slopes and intercepts
    print(f"RAP < 0 Fit: Slope = {model1.coef_[0]:.2f}, Intercept = {model1.intercept_:.2f}")
    print(f"0 ≤ RAP ≤ 0.4 Fit: Slope = {model2.coef_[0]:.2f}, Intercept = {model2.intercept_:.2f}")
    print(f"RAP > 0.4 Fit: Slope = {model3.coef_[0]:.2f}, Intercept = {model3.intercept_:.2f}")
    
    return round(model1.coef_[0], 2), round(model2.coef_[0], 2), round(model3.coef_[0], 2)

In [ ]:
# For a single parameter
dependent_param = 'ICP'
resolution = 'H'
slope_1, slope_2, slope_3 = scatter_plot_with_piece_wise_linear_regression(df_all, dependent_param, resolution, filtered_timestamps)

In [ ]:
resolution = 'H'
labels = ['RAP < 0', '0 ≤ RAP ≤ 0.4', 'RAP > 0.4']
columns = df_all.columns.to_list()
columns.remove('RAP')
columns.remove('DateTime')
df_slope = pd.DataFrame(index=range(len(columns)), columns=labels)

for index, parameter in enumerate(columns):
    slope_1, slope_2, slope_3 = scatter_plot_with_piece_wise_linear_regression(df_all, parameter, resolution, filtered_timestamps)

    df_slope.at[index, labels[0]] = slope_1
    df_slope.at[index, labels[1]] = slope_2
    df_slope.at[index, labels[2]] = slope_3
    
df_slope.index = columns

In [ ]:
df_slope

In [ ]:
resolution = 'D'
labels = ['RAP < 0', '0 ≤ RAP ≤ 0.4', 'RAP > 0.4']
columns = df_all.columns.to_list()
columns.remove('RAP')
columns.remove('DateTime')
df_slope = pd.DataFrame(index=range(len(columns)), columns=labels)

for index, parameter in enumerate(columns):
    slope_1, slope_2, slope_3 = scatter_plot_with_piece_wise_linear_regression(df_all, parameter, resolution, filtered_timestamps)

    df_slope.at[index, labels[0]] = slope_1
    df_slope.at[index, labels[1]] = slope_2
    df_slope.at[index, labels[2]] = slope_3
    
df_slope.index = columns

In [ ]:
df_slope

In [ ]:
resolution = 'M'
labels = ['RAP < 0', '0 ≤ RAP ≤ 0.4', 'RAP > 0.4']
columns = df_all.columns.to_list()
columns.remove('RAP')
columns.remove('DateTime')
df_slope = pd.DataFrame(index=range(len(columns)), columns=labels)

for index, parameter in enumerate(columns):
    slope_1, slope_2, slope_3 = scatter_plot_with_piece_wise_linear_regression(df_all, parameter, resolution, filtered_timestamps)

    df_slope.at[index, labels[0]] = slope_1
    df_slope.at[index, labels[1]] = slope_2
    df_slope.at[index, labels[2]] = slope_3
    
df_slope.index = columns

In [ ]:
df_slope

## Boxplots

In [ ]:
def boxplots_with_values(df_all, dependent_param, values, resolution, filtered_timestamps):
    
    warnings.filterwarnings('ignore')
     
    # Separating the columns that would be plotted
    df_pair = df_all[['DateTime', 'RAP', dependent_param]]
    
    # Declaring the NIRS measures
    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']
    # Checking if it's a NIRS measure and if yes, dropping the rows according to the threshold
    if dependent_param in NIRS_measures_list:
        df_pair = df_pair[~df_pair['DateTime'].isin(filtered_timestamps)]
        df_pair = df_pair.loc[df_pair[dependent_param] != 0]
        
    # Dropping the Nan values
    df_pair = df_pair.dropna()
    # Converting the resolution
    if resolution == 'M':
        df_pair_resolution = df_pair
    else:   
        df_pair_resolution = convert_parameter_resolution(df_pair, resolution)
        
    labels = []
    segments = []
    
    for index in range(len(values)-1):
        label = "[" + str(values[index]) + "," + str(values[index+1]) + ")"
        labels.append(label)
        segment = df_pair_resolution[(df_pair_resolution['RAP'] >= values[index]) & (df_pair_resolution['RAP'] < values[index+1])]
        segments.append(segment[dependent_param].dropna())
    
    plt.figure(figsize=(10, 8))
    plt.boxplot(segments, labels=labels, showfliers=False)
    
    title = dependent_param + ' Measures at Different RAP Values'
    plt.title(title)
    plt.xticks(fontsize=8)
    plt.xlabel("RAP")
    plt.ylabel(dependent_param)
    plt.axvline(x=5.5, color='black', linestyle='--', linewidth=1)  # Approx middle of 0 and 0.2
    plt.axvline(x=7.5, color='black', linestyle='--', linewidth=1)  # Approx middle of 0.4 and 0.6

    plt.tight_layout()
    
    save_image_directory = r"D:\Thesis Project\Thesis Project 2\Research Phase 4\Plots\Boxplots_v2\RAP_independent"
    save_image_name = resolution + ' RAP and ' + dependent_param
    plt.savefig(os.path.join(save_image_directory, save_image_name))
    
    plt.show()

In [ ]:
values = np.round(np.arange(-1, 1.1, 0.2), 1)
values = np.where(values == -0.0, 0.0, values)

columns = df_all.columns.to_list()
columns.remove('RAP')
columns.remove('DateTime')

resolution = "H"

for index, parameter in enumerate(columns):
    boxplots_with_values(df_all, parameter, values, resolution, filtered_timestamps)

In [ ]:
values = np.round(np.arange(-1, 1.1, 0.2), 1)
values = np.where(values == -0.0, 0.0, values)

columns = df_all.columns.to_list()
columns.remove('RAP')
columns.remove('DateTime')

resolution = "D"

for index, parameter in enumerate(columns):
    boxplots_with_values(df_all, parameter, values, resolution, filtered_timestamps)

In [ ]:
values = np.round(np.arange(-1, 1.1, 0.2), 1)
values = np.where(values == -0.0, 0.0, values)

columns = df_all.columns.to_list()
columns.remove('RAP')
columns.remove('DateTime')

resolution = "M"

for index, parameter in enumerate(columns):
    boxplots_with_values(df_all, parameter, values, resolution, filtered_timestamps)

## Contour Plots

In [ ]:
def contour_plot_across_full_spectrum(breakpoints, dependent_param):
    
    tic = timer()
    warnings.filterwarnings('ignore')
    
    segment = df_all[['RAP', 'ICP', dependent_param]].dropna()
    
    # Extract X, Y, and Z columns
    x = segment['RAP']
    y = segment['ICP']
    z = segment[dependent_param]
    
    # Create grid data for contour plot
    grid_x, grid_y = np.meshgrid(
        np.linspace(x.min(), x.max(), 100),
        np.linspace(y.min(), y.max(), 100)
    )

    # Interpolate Z values on the grid
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='linear')

    # Plot the contour
    plt.figure(figsize=(8, 6))
    contour = plt.contourf(grid_x, grid_y, grid_z, levels=20, cmap='viridis')
    plt.colorbar(contour, label=dependent_param)
    
    plt.axvline(x=breakpoints[0], color='black', linestyle='--', linewidth=2)  
    plt.axvline(x=breakpoints[1], color='black', linestyle='--', linewidth=2)
    
    title = 'Contour Plot of ' + dependent_param
    plt.title(title)
    plt.xlabel('RAP')
    plt.ylabel('ICP')

    save_image_directory = r"D:\Thesis Project\Thesis Project 2\Research Phase 4\Plots\Contour_plots\RAP_independent"
    save_image_name = 'RAP and ' + dependent_param
    plt.savefig(os.path.join(save_image_directory, save_image_name))

    plt.show()
    
    tac = timer()
    print("Execution time:", round(tac-tic, 2), "seconds")

In [ ]:
# For all the parameters

breakpoints = [0, 0.4]

for index, parameter in enumerate(df_all.columns):
    if index == 0 or index == 1:           # excluding RAP and ICP 
        pass
    else:
        contour_plot_across_full_spectrum(breakpoints, parameter)

# Median Physiology Measures

In [ ]:
# Splitting data into three segments

breakpoints = [0, 0.4]

segment1 = df_all[df_all['RAP'] < breakpoints[0]]
segment2 = df_all[(df_all['RAP'] >= breakpoints[0]) & (df_all['RAP'] <= breakpoints[1])]
segment3 = df_all[df_all['RAP'] > breakpoints[1]]

segments = [segment1, segment2, segment3]

labels = ['RAP < 0', '0 ≤ RAP ≤ 0.4', 'RAP > 0.4']

In [ ]:
# Median value of each physiology measure
# Defining the dataframe
num_rows = len(labels)
columns = df_all.columns.to_list()
columns.insert(0, 'Labels')
columns.remove('DateTime')

df_median = pd.DataFrame(index=range(num_rows), columns=columns)
abs(max_lags).quantile(0.75)
# Insert median values
for index, segment in enumerate(segments):
    df_median.loc[index] = segment.median().round(3)

df_median['Labels'] = labels
df_median = df_median.set_index("Labels")
df_median.T

In [ ]:
# Median with IQR
def median_iqr(df_parameter):
    median = round(df_parameter.median(), 3)
    q3 = round(df_parameter.quantile(0.75), 3)
    q1 = round(df_parameter.quantile(0.25), 3)
    return str(median) + '(' + str(q1) + '\u2014' + str(q3) + ')'

In [ ]:
# Median with IQR
parameter_list = ['DateTime', 'RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                          'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
parameter_list_modified = ['RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                          'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

df_median = pd.DataFrame(index=labels, columns=parameter_list_modified)

for segment_no, segment in enumerate(segments):
    print("Segment in progress: ", labels[segment_no])
    for parameter_index, parameter in enumerate(parameter_list_modified):
        #print("Parameter in progress: ", parameter)
        if parameter in NIRS_measures_list:
            segment_modified = cleaning_NIRS_measures_parameters_v2(segment, parameter)
        else:
            segment_modified = segment
        
        df_parameter = segment_modified[parameter]
        df_median.at[labels[segment_no], parameter] = median_iqr(df_parameter)
    
    print("\n")
    
df_median.T

In [ ]:
# Median value of each physiology measure
# Defining the dataframe
columns = df_all.columns.to_list()
columns.remove('DateTime')
df_median = pd.DataFrame(index=labels, columns=columns)

NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

# Insert median values
for segment_no, segment in enumerate(segments):
    for column_no, parameter in enumerate(columns):
        if parameter in NIRS_measures_list:
            df_pair = segment[['DateTime', parameter]]
            df_pair = df_pair[~df_pair['DateTime'].isin(filtered_timestamps)]
            df_pair = df_pair.loc[df_pair[parameter] != 0]
            df_median.at[labels[segment_no], parameter] = round(df_pair[parameter].median(), 3)
        else:
            df_median.at[labels[segment_no], parameter] = round(segment[parameter].median(), 3) 

df_median.T

## Jonckheere-Terpstra testing

In [ ]:
save_directory = r"D:\Thesis Project\Thesis Project 2\Research Phase 4\Data\Segment_csv\RAP_independent"

for index, segment in enumerate(segments):
    file_name = "segment" + str(index+1) + ".csv"
    segment.to_csv(os.path.join(save_directory, file_name), index=False)

In [ ]:
# The rest is conducted in R

## Kruskal-Wallis testing

In [ ]:
def read_personalized_data(file, parameter_list, resolution):
    
    df_individual= pd.read_csv(os.path.join(csv_file_directory, file))
    df_individual_cleaned = df_individual.reindex(columns = parameter_list)
    #df_individual_cleaned = df_individual_cleaned.dropna(axis=1, how='all')
    filter_condition = (df_individual_cleaned['ICP'] > 100) | (df_individual_cleaned['ICP'] < 0) | \
                           (df_individual_cleaned['MAP'] > 200) | (df_individual_cleaned['MAP'] < 0) | \
                           (df_individual_cleaned['AMP'] > 40)
    df_individual_cleaned = df_individual_cleaned[~filter_condition]
    
 # Converting resolution of the data if necessary
    if resolution == 'M':
        pass
    else:   
        df_individual_cleaned = convert_parameter_resolution(df_individual_cleaned, resolution)
        
    return df_individual_cleaned

In [ ]:
def median_from_each_patient():
    
    parameter_list = ['DateTime', 'RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                          'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    parameter_list_modified = ['RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                              'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

    
    df_all_median = pd.DataFrame()
    resolution = 'M'
    files = os.listdir(csv_file_directory)

    for file in files:
        df_median = pd.DataFrame(index=[0], columns=parameter_list)
        df_individual_cleaned = read_personalized_data(file, parameter_list, resolution)
        for parameter_index, parameter in enumerate(parameter_list):
            if parameter in NIRS_measures_list:
                df_individual_cleaned_modified = cleaning_NIRS_measures_parameters_v2(df_individual_cleaned, parameter)
            else:
                df_individual_cleaned_modified = df_individual_cleaned

            df_median.at[0, parameter] = df_individual_cleaned_modified[parameter].median()

        df_all_median = pd.concat([df_all_median, df_median], axis=0, ignore_index=True)
        
    return df_all_median

In [ ]:
df_all_median = median_from_each_patient()

In [ ]:
# Splitting data into three segments

breakpoints = [0, 0.4]

segment1 = df_all_median[df_all_median['RAP'] < breakpoints[0]]
segment2 = df_all_median[(df_all_median['RAP'] >= breakpoints[0]) & (df_all_median['RAP'] <= breakpoints[1])]
segment3 = df_all_median[df_all_median['RAP'] > breakpoints[1]]

segments = [segment1, segment2, segment3]

labels = ['RAP < 0', '0 ≤ RAP ≤ 0.4', 'RAP > 0.4']

In [ ]:
segment3.shape

In [ ]:
# With median from each patient

parameter_list = ['DateTime', 'RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                          'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
parameter_list_modified = ['RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                          'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

df_comparison = pd.DataFrame(index=range(2), columns=parameter_list_modified)

for parameter in parameter_list_modified:
    
    stat, p_value = kruskal(segment1[parameter].dropna(), segment2[parameter].dropna(), segment3[parameter].dropna())
    df_comparison.at[0, parameter] = stat
    df_comparison.at[1, parameter] = p_value

df_comparison = df_comparison.applymap(lambda x: "{:.3f}".format(x) if x >= 0.001 else "{:.2e}".format(x))
df_comparison.T

In [ ]:
columns = df_all.columns.to_list()
columns.remove('DateTime')

df_comparison = pd.DataFrame(index=range(2), columns=columns)

for parameter in columns:
    if parameter in NIRS_measures_list:
        group_1 = segments[0][['DateTime', parameter]]
        group_1 = group_1[~group_1['DateTime'].isin(filtered_timestamps)]
        group_1 = group_1.loc[group_1[parameter] != 0]

        group_2 = segments[1][['DateTime', parameter]]
        group_2 = group_2[~group_2['DateTime'].isin(filtered_timestamps)]
        group_2 = group_2.loc[group_2[parameter] != 0]

        group_3 = segments[2][['DateTime', parameter]]
        group_3 = group_3[~group_3['DateTime'].isin(filtered_timestamps)]
        group_3 = group_3.loc[group_3[parameter] != 0]
        
    else:
        group_1 = segments[0][['DateTime', parameter]]
        group_2 = segments[1][['DateTime', parameter]]
        group_3 = segments[2][['DateTime', parameter]]
            
    stat, p_value = kruskal(group_1[parameter].dropna(), group_2[parameter].dropna(), group_3[parameter].dropna())
    df_comparison.at[0, parameter] = stat
    df_comparison.at[1, parameter] = p_value

df_comparison = df_comparison.applymap(lambda x: "{:.2e}".format(x))
df_comparison.T

# Reverse with Other Physiological Thresholds

## Median

In [ ]:
def median_with_other_physiologic_thresholds(segments):
    # Median value of each physiology measure
    # Defining the dataframe
    columns = df_all.columns.to_list()
    columns.remove('DateTime')
    df_median = pd.DataFrame(index=range(len(segments)), columns=columns)

    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

    # Insert median values
    for segment_no, segment in enumerate(segments):
        for column_no, parameter in enumerate(columns):
            if parameter in NIRS_measures_list:
                df_pair = segment[['DateTime', parameter]]
                df_pair = df_pair[~df_pair['DateTime'].isin(filtered_timestamps)]
                df_pair = df_pair.loc[df_pair[parameter] != 0]
                df_median.at[segment_no, parameter] = round(df_pair[parameter].median(), 3)
            else:
                df_median.at[segment_no, parameter] = round(segment[parameter].median(), 3) 

    return df_median.T

In [ ]:
# Median with IQR
def median_with_iqr_with_other_physiologic_thresholds(segments):
    
    parameter_list = ['DateTime', 'RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                          'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    parameter_list_modified = ['RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                              'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

    df_median = pd.DataFrame(index=range(len(segments)), columns=parameter_list_modified)

    for segment_no, segment in enumerate(segments):
        print("Segment in progress: ", segment_no)
        for parameter_index, parameter in enumerate(parameter_list_modified):
            #print("Parameter in progress: ", parameter)
            if parameter in NIRS_measures_list:
                segment_modified = cleaning_NIRS_measures_parameters_v2(segment, parameter)
            else:
                segment_modified = segment

            df_parameter = segment_modified[parameter]
            df_median.at[segment_no, parameter] = median_iqr(df_parameter)
            
    return df_median.T

## ICP

In [ ]:
# Calculating the segments, threshold 20
independent_param = "ICP"
breakpoints = [20]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

In [ ]:
# Calculating the segments, threshold 22.5
independent_param = "ICP"
breakpoints = [22.5]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## CPP

In [ ]:
# Calculating the segments, threshold below 60 and above 70
independent_param = "CPP"
breakpoints = [60, 70]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## rSO2

In [ ]:
independent_param = "RSO2_L"
breakpoints = [60, 70, 80, 90]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]
segment3 = all_data[all_data[independent_param] < breakpoints[2]]
segment4 = all_data[all_data[independent_param] < breakpoints[3]]

segments = [segment1, segment2, segment3, segment4]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

In [ ]:
independent_param = "RSO2_R"
breakpoints = [60, 70, 80, 90]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]
segment3 = all_data[all_data[independent_param] < breakpoints[2]]
segment4 = all_data[all_data[independent_param] < breakpoints[3]]

segments = [segment1, segment2, segment3, segment4]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## PbtO2

In [ ]:
independent_param = "PbtO2"
breakpoints = [15, 20]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0], breakpoints[1])
df_median

## PRx

In [ ]:
independent_param = "PRx"
breakpoints = [0, 0.25, 0.35]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]
segment3 = all_data[all_data[independent_param] > breakpoints[2]]

segments = [segment1, segment2, segment3]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## PAx

In [ ]:
independent_param = "PAx"
breakpoints = [0, 0.25]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## RAC

In [ ]:
independent_param = "RAC"
breakpoints = [0]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[0]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## COx

In [ ]:
independent_param = "COx_L"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

In [ ]:
independent_param = "COx_R"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## COx-a

In [ ]:
independent_param = "COxA_L"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

In [ ]:
independent_param = "COxA_R"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_median = median_with_iqr_with_other_physiologic_thresholds(segments)
print("Median based on threshold", breakpoints[0])
df_median

## Jonckheere-Terpstra testing

## rSO2

In [ ]:
independent_param = "RSO2_L"
breakpoints = [60, 70, 80, 90]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]
segment3 = all_data[all_data[independent_param] < breakpoints[2]]
segment4 = all_data[all_data[independent_param] < breakpoints[3]]

segments = [segment1, segment2, segment3, segment4]

save_directory = r"D:\Thesis Project\Thesis Project 2\Research Phase 4\Segment_csv\Other_variables_independent"

for index, segment in enumerate(segments):
    file_name = independent_param + "_segment" + str(index+1) + ".csv"
    segment.to_csv(os.path.join(save_directory, file_name), index=False)

In [ ]:
# The Rest is done in R

## Kruskal-Wallis testing

In [ ]:
def kruskal_wallis_testing_result(segments):
        
    columns = df_all.columns.to_list()
    columns.remove('DateTime')

    df_comparison = pd.DataFrame(index=range(2), columns=columns)
    
    for parameter in columns:
        groups = []
        if parameter in NIRS_measures_list:
            for i in range(len(segments)):
                group = segments[i][['DateTime', parameter]]
                group = group[~group['DateTime'].isin(filtered_timestamps)]
                group = group.loc[group[parameter] != 0]
                groups.append(group[parameter].dropna())

        else:
            for i in range(len(segments)):
                group = segments[i][['DateTime', parameter]]
                groups.append(group[parameter].dropna())

        stat, p_value = kruskal(*groups)
        df_comparison.at[0, parameter] = stat
        df_comparison.at[1, parameter] = p_value

    df_comparison = df_comparison.applymap(lambda x: "{:.2e}".format(x))
            
    return df_comparison.T

In [ ]:
# With median from each patient
def kruskal_wallis_testing_result_v2(segments):
    
    parameter_list = ['DateTime', 'RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                              'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    parameter_list_modified = ['RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                              'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

    df_comparison = pd.DataFrame(index=range(2), columns=parameter_list_modified)
    
    for parameter in parameter_list_modified:
        
        segments_cleaned = [segment[parameter].dropna() for segment in segments]

        stat, p_value = kruskal(*segments_cleaned)
        df_comparison.at[0, parameter] = stat
        df_comparison.at[1, parameter] = p_value

    df_comparison = df_comparison.applymap(lambda x: "{:.3f}".format(x) if x >= 0.001 else "{:.2e}".format(x))
    return df_comparison.T

In [ ]:
independent_param = "RSO2_L"
breakpoints = [60, 70, 80, 90]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]
segment3 = all_data[all_data[independent_param] < breakpoints[2]]
segment4 = all_data[all_data[independent_param] < breakpoints[3]]

segments = [segment1, segment2, segment3, segment4]

df_comparison = kruskal_wallis_testing_result_v2(segments)
df_comparison

In [ ]:
independent_param = "RSO2_R"
breakpoints = [60, 70, 80, 90]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]
segment3 = all_data[all_data[independent_param] < breakpoints[2]]
segment4 = all_data[all_data[independent_param] < breakpoints[3]]

segments = [segment1, segment2, segment3, segment4]

df_comparison = kruskal_wallis_testing_result_v2(segments)
df_comparison

In [ ]:
# The Rest is done in R

## PRx

In [ ]:
independent_param = "PRx"
breakpoints = [0, 0.25, 0.35]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]
segment3 = all_data[all_data[independent_param] > breakpoints[2]]

segments = [segment1, segment2, segment3]

df_comparison = kruskal_wallis_testing_result_v2(segments)
df_comparison

In [ ]:
# The Rest is done in R

## Mann_Whitney_U_Test

In [ ]:
def mann_whitney_U_test_result(segments):
    
    df_comparison = pd.DataFrame(index=range(2), columns=segments[0].columns)

    for parameter in segments[0].columns:
        stat, p_value = mannwhitneyu(segments[0][parameter].dropna(), segments[1][parameter].dropna())
        df_comparison.at[0, parameter] = stat
        df_comparison.at[1, parameter] = p_value
        
    df_comparison = df_comparison.applymap(lambda x: "{:.2e}".format(x))
    
    return df_comparison

In [ ]:
def mann_whitney_U_test_result_v2(segments):
    
    parameter_list = ['DateTime', 'RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                              'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    parameter_list_modified = ['RAP', 'ICP', 'AMP', 'MAP', 'CPP', 'PRx', 'PAx', 'RAC', \
                              'RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R', 'PbtO2']
    NIRS_measures_list = ['RSO2_L', 'RSO2_R', 'COx_L', 'COx_R', 'COxA_L', 'COxA_R']

    df_comparison = pd.DataFrame(index=range(2), columns=parameter_list_modified)
    
    for parameter in parameter_list_modified:
        
        #segments_cleaned = [segment[parameter].dropna() for segment in segments]
        
        x = pd.to_numeric(segments[0][parameter], errors='coerce').dropna()
        y = pd.to_numeric(segments[1][parameter], errors='coerce').dropna()

        stat, p_value = mannwhitneyu(x, y)
        df_comparison.at[0, parameter] = stat
        df_comparison.at[1, parameter] = p_value

    df_comparison = df_comparison.applymap(lambda x: "{:.3f}".format(x) if x >= 0.001 else "{:.2e}".format(x))
    return df_comparison.T

In [ ]:
def mann_whitney_U_test_result(segments):
        
    columns = df_all.columns.to_list()
    columns.remove('DateTime')

    df_comparison = pd.DataFrame(index=range(2), columns=columns)
    
    
    for parameter in columns:
        groups = []
        if parameter in NIRS_measures_list:
            for i in range(len(segments)):
                group = segments[i][['DateTime', parameter]]
                group = group[~group['DateTime'].isin(filtered_timestamps)]
                group = group.loc[group[parameter] != 0]
                groups.append(group[parameter].dropna())

        else:
            for i in range(len(segments)):
                group = segments[i][['DateTime', parameter]]
                groups.append(group[parameter].dropna())
        
        stat, p_value= mannwhitneyu(groups[0], groups[1])
        #print(stat)
        df_comparison.at[0, parameter] = stat
        df_comparison.at[1, parameter] = p_value

    df_comparison = df_comparison.applymap(lambda x: "{:.2e}".format(x))
            
    return df_comparison.T

In [ ]:
columns = df_all.columns.to_list()
columns.remove('DateTime')
df_comparison = pd.DataFrame(index=range(2), columns=columns)
    
for parameter in columns:
    groups = []
    if parameter in NIRS_measures_list:
        for i in range(len(segments)):
            group = segments[i][['DateTime', parameter]]
            group = group[~group['DateTime'].isin(filtered_timestamps)]
            group = group.loc[group[parameter] != 0]
            groups.append(group[parameter].dropna())

    else:
        for i in range(len(segments)):
            group = segments[i][['DateTime', parameter]]
            groups.append(group[parameter].dropna())

    stat, p_value = mannwhitneyu(groups[0], groups[1])

In [ ]:
NIRS_measures_list

## ICP

In [ ]:
# Calculating the segments, threshold 20
independent_param = "ICP"
breakpoints = [20]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

In [ ]:
# Calculating the segments, threshold 22.5
independent_param = "ICP"
breakpoints = [22.5]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

## CPP

In [ ]:
# Calculating the segments, threshold below 60 and above 70
independent_param = "CPP"
breakpoints = [60, 70]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

## PbtO2

In [ ]:
independent_param = "PbtO2"
breakpoints = [15, 20]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

## PAx

In [ ]:
independent_param = "PAx"
breakpoints = [0, 0.25]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

## RAC

In [ ]:
independent_param = "RAC"
breakpoints = [0]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[0]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

## COx

In [ ]:
independent_param = "COx_L"
breakpoints = [0, 0.20]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

In [ ]:
independent_param = "COx_R"
breakpoints = [0, 0.20]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

## COx-a

In [ ]:
independent_param = "COxA_L"
breakpoints = [0, 0.20]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

In [ ]:
independent_param = "COxA_R"
breakpoints = [0, 0.20]
all_data = df_all_median

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

In [ ]:
df_comparison = mann_whitney_U_test_result_v2(segments)
df_comparison

## %Time with RAP Above Thresholds

In [ ]:
def time_above_threshold(segments):
    
    # Defining an empty dataframe to save the %time result
    num_rows = len(segments)
    columns = ['0 ≤ RAP ≤ 0.4 (%)','RAP > 0.4 (%)', 'RAP < 0 (%)']
    df_percentage_time = pd.DataFrame(index=range(num_rows), columns=columns)
    
    for segment_index, segment in enumerate(segments):
        
        # Calculating total time
        df_RAP = segment['RAP'].dropna()
        total_time = df_RAP.shape[0]
        
        # Separating the RAP states at each segment
        RAP_state_1 = df_RAP[(df_RAP >= 0) & (df_RAP <= 0.4)]
        RAP_state_2 = df_RAP[df_RAP > 0.4]
        RAP_state_3 = df_RAP[df_RAP < 0]
        
        RAP_states = [RAP_state_1, RAP_state_2, RAP_state_3]
        
        # Calculating the time spent for different states at each segment
        for state_index, RAP_state in enumerate(RAP_states):
            time_spent_state = RAP_state.shape[0]
            percentage_time_with_RAP_above_threshold = (time_spent_state/total_time) * 100
            
            # Insert percentage time values
            df_percentage_time.iloc[segment_index, state_index] = round(percentage_time_with_RAP_above_threshold, 3)
        
    return df_percentage_time

In [ ]:
def time_above_threshold_v2(segments, parameter):
    
    # Defining an empty dataframe to save the %time result
    num_rows = len(segments)
    #columns = ['0 ≤ RAP ≤ 0.4 (%)','RAP > 0.4 (%)', 'RAP < 0 (%)']
    df_percentage_time = pd.DataFrame(index=range(len(segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        
        # Calculating total time
        df_parameter = segment[parameter].dropna()
        total_time = df_RAP.shape[0]
        
        # Separating the RAP states at each segment
        RAP_state_1 = df_RAP[(df_RAP >= 0) & (df_RAP <= 0.4)]
        RAP_state_2 = df_RAP[df_RAP > 0.4]
        RAP_state_3 = df_RAP[df_RAP < 0]
        
        RAP_states = [RAP_state_1, RAP_state_2, RAP_state_3]
        
        # Calculating the time spent for different states at each segment
        for state_index, RAP_state in enumerate(RAP_states):
            time_spent_state = RAP_state.shape[0]
            percentage_time_with_RAP_above_threshold = (time_spent_state/total_time) * 100
            
            # Insert percentage time values
            df_percentage_time.iloc[segment_index, state_index] = round(percentage_time_with_RAP_above_threshold, 3)
        
    return df_percentage_time

In [ ]:
# Splitting data into three segments

breakpoints = [0, 0.4]
all_data = df_all_median

segment1 = all_data[all_data['RAP'] < breakpoints[0]]
segment2 = all_data[(all_data['RAP'] >= breakpoints[0]) & (all_data['RAP'] <= breakpoints[1])]
segment3 = all_data[all_data['RAP'] > breakpoints[1]]

RAP_segments = [segment1, segment2, segment3]

In [ ]:
df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))

for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    independent_param = "ICP"
    breakpoints = [20]
    all_data = RAP_segment
    total_time = RAP_segment.shape[0]

    segment1 = all_data[all_data[independent_param] < breakpoints[0]]
    segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

    segments = [segment1, segment2]
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
# Calculating the segments, threshold 20

independent_param = "ICP"
breakpoints = [20]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
# Calculating the segments, threshold 22.5

independent_param = "ICP"
breakpoints = [22.5]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
# Calculating the segments, threshold below 60, above 70

independent_param = "CPP"
breakpoints = [60, 70]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "RSO2_L"
breakpoints = [60, 70, 80, 90]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]
segment3 = all_data[all_data[independent_param] < breakpoints[2]]
segment4 = all_data[all_data[independent_param] < breakpoints[3]]

segments = [segment1, segment2, segment3, segment4]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "RSO2_R"
breakpoints = [60, 70, 80, 90]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]
segment3 = all_data[all_data[independent_param] < breakpoints[2]]
segment4 = all_data[all_data[independent_param] < breakpoints[3]]

segments = [segment1, segment2, segment3, segment4]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "PbtO2"
breakpoints = [15, 20]
all_data = df_all

segment1 = all_data[all_data[independent_param] < breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[1]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "PRx"
breakpoints = [0, 0.25, 0.35]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]
segment3 = all_data[all_data[independent_param] > breakpoints[2]]

segments = [segment1, segment2, segment3]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "PAx"
breakpoints = [0, 0.25]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "RAC"
breakpoints = [0]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] < breakpoints[0]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "COx_L"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "COx_R"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "COxA_L"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

In [ ]:
independent_param = "COxA_R"
breakpoints = [0, 0.20]
all_data = df_all

segment1 = all_data[all_data[independent_param] > breakpoints[0]]
segment2 = all_data[all_data[independent_param] > breakpoints[1]]

segments = [segment1, segment2]

df_percentage_time = time_above_threshold(segments=segments)
df_percentage_time

## Reverse %time spent

In [ ]:
# Splitting data into three segments

breakpoints = [0, 0.4]
all_data = df_all_median

segment1 = all_data[all_data['RAP'] < breakpoints[0]]
segment2 = all_data[(all_data['RAP'] >= breakpoints[0]) & (all_data['RAP'] <= breakpoints[1])]
segment3 = all_data[all_data['RAP'] > breakpoints[1]]

RAP_segments = [segment1, segment2, segment3]

In [ ]:
df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))

for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    independent_param = "ICP"
    breakpoints = [20]
    all_data = RAP_segment
    total_time = RAP_segment.shape[0]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)

    segment1 = all_data[all_data[independent_param] < breakpoints[0]]
    segment2 = all_data[all_data[independent_param] >= breakpoints[0]]

    segments = [segment1, segment2]
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))

for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "ICP"
    breakpoints = [22.5]
    all_data = RAP_segment
    segment1 = all_data[all_data[independent_param] < breakpoints[0]]
    segment2 = all_data[all_data[independent_param] >= breakpoints[0]]
    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "CPP"
    breakpoints = [60, 70]
    all_data = RAP_segment
    segment1 = all_data[all_data[independent_param] < breakpoints[0]]
    segment2 = all_data[all_data[independent_param] > breakpoints[1]]
    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "PRx"
    breakpoints = [0, 0.25, 0.35]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] > breakpoints[0]]
    segment2 = all_data[all_data[independent_param] > breakpoints[1]]
    segment3 = all_data[all_data[independent_param] > breakpoints[2]]

    segments = [segment1, segment2, segment3]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "PAx"
    breakpoints = [0, 0.25]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] > breakpoints[0]]
    segment2 = all_data[all_data[independent_param] > breakpoints[1]]

    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "RAC"
    breakpoints = [0]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] > breakpoints[0]]
    segment2 = all_data[all_data[independent_param] < breakpoints[0]]

    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "RSO2_L"
    breakpoints = [60, 70, 80, 90]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] < breakpoints[0]]
    segment2 = all_data[all_data[independent_param] < breakpoints[1]]
    segment3 = all_data[all_data[independent_param] < breakpoints[2]]
    segment4 = all_data[all_data[independent_param] < breakpoints[3]]

    segments = [segment1, segment2, segment3, segment4]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "RSO2_R"
    breakpoints = [60, 70, 80, 90]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] < breakpoints[0]]
    segment2 = all_data[all_data[independent_param] < breakpoints[1]]
    segment3 = all_data[all_data[independent_param] < breakpoints[2]]
    segment4 = all_data[all_data[independent_param] < breakpoints[3]]

    segments = [segment1, segment2, segment3, segment4]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "COx_L"
    breakpoints = [0, 0.20]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] > breakpoints[0]]
    segment2 = all_data[all_data[independent_param] > breakpoints[1]]

    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "COx_R"
    breakpoints = [0, 0.20]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] > breakpoints[0]]
    segment2 = all_data[all_data[independent_param] > breakpoints[1]]

    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "COxA_L"
    breakpoints = [0, 0.20]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] > breakpoints[0]]
    segment2 = all_data[all_data[independent_param] > breakpoints[1]]

    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "COxA_R"
    breakpoints = [0, 0.20]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] > breakpoints[0]]
    segment2 = all_data[all_data[independent_param] > breakpoints[1]]

    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time

In [ ]:
for RAP_segment_no, RAP_segment in enumerate(RAP_segments):
    
    total_time = RAP_segment.shape[0]
    
    independent_param = "PbtO2"
    breakpoints = [15, 20]
    all_data = RAP_segment

    segment1 = all_data[all_data[independent_param] < breakpoints[0]]
    segment2 = all_data[all_data[independent_param] < breakpoints[1]]

    segments = [segment1, segment2]
    
    if RAP_segment_no == 0:
        print("Parameter in progress: ", independent_param)
        df_percentage_time = pd.DataFrame(index=range(len(RAP_segments)), columns=range(len(segments)))
    
    for segment_index, segment in enumerate(segments):
        time_spent_state = segment.shape[0]
        percentage_time_above_threshold = (time_spent_state/total_time) * 100
        #Insert percentage time values
        df_percentage_time.at[RAP_segment_no, segment_index] = round(percentage_time_above_threshold, 3)
        
df_percentage_time